# UP3-CNN

# 📚 Libraries

In [ ]:
import os
import math
import wave
import joblib
import librosa
import pylab
import pandas as pd
import numpy as np
import soundfile as sf
import matplotlib
matplotlib.use('Agg')  # Avoid GUI backend
import librosa.display
import matplotlib.pyplot as plt

# 📥 FUNCIONES UTILITARIAS

In [ ]:
# 📄 Obtener información de un archivo de audio
def obtener_info_audio(archivo_wav):
    info = sf.info(archivo_wav)
    tasa_muestreo = info.samplerate
    subtipo = info.subtype

    if 'PCM' in subtipo:
        try:
            bits = int(subtipo.split('_')[1])
        except (IndexError, ValueError):
            bits = "No disponible"
    else:
        bits = "No disponible"

    return tasa_muestreo, bits

# 🖼️ Generar espectrograma Mel

In [ ]:
def generar_mel_spectrogram(audio_path, output_path, sample_rate=22050):
    try:
        # Cargar y escribir en formato WAV por compatibilidad
        audio_data, sr = sf.read(audio_path)
        sf.write(audio_path, audio_data, sr, format='WAV')

        # Leer con wave y convertir a array
        with wave.open(audio_path) as audio:
            sr = audio.getframerate()
            audio_data = np.frombuffer(audio.readframes(-1), dtype='int16')

        # Parámetros para el espectrograma
        n_fft = 2048
        hop_length = 32
        n_mels = 128
        fmin = 0
        fmax = sr // 2

        S = librosa.feature.melspectrogram(
            y=audio_data.astype(np.float32),
            sr=sr,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
            fmin=fmin,
            fmax=fmax
        )

        # Crear figura sin bordes
        fig = plt.figure(1, frameon=False)
        fig.set_size_inches(6, 6)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)

        librosa.display.specshow(librosa.power_to_db(S, ref=np.max), fmin=fmin, y_axis='linear')
        fig.savefig(output_path)
        plt.close(fig)

        print(f"✅ Mel spectrogram saved: {output_path}")
    except Exception as e:
        print(f"❌ Error processing {audio_path}: {e}")

# 📁 CREACIÓN DE DIRECTORIOS DE CLASES

In [ ]:
def crear_directorios_mels(csv_path, output_dir):
    csv_data = pd.read_csv(csv_path)
    clases = csv_data['class'].unique()
    for clase in clases:
        ruta_clase = os.path.join(output_dir, clase)
        os.makedirs(ruta_clase, exist_ok=True)
    return csv_data

# 🧠 MAIN SCRIPT

In [ ]:
if __name__ == "__main__":
    # 🔍 Verificación de información de audio
    ejemplos = [
        "C:/Users/cadur/Downloads/Urbansonic/0_Mels_Gen/DATA/Urbanphony/PPN/Punto 01 P1.wav",  # PPN
        "C:/Users/cadur/Downloads/Urbansonic/0_Mels_Gen/DATA/Urbanphony/SQC/JAN_1_P1.wav",     # SQC
        "C:/Users/cadur/Downloads/Urbansonic/0_Mels_Gen/DATA/Urbanphony/VNE/1 R.wav"          # VNE
    ]

    print("🔎 Información de audios:")
    for archivo in ejemplos:
        tasa, bits = obtener_info_audio(archivo)
        print(f"{archivo}\n  📈 Tasa: {tasa} Hz, 🧊 Bits: {bits}")

    # 📍 Paths
    wav_dir = 'E:/VNE/Audios/Audios_VNE_2s/VNE_2seg'
    mel_spec_dir = "E:/VNE/Mels_Completo/Mels_New"
    csv_path = "E:/VNE/CSV/VNE_AllMels.csv"

    # 🗂️ Crear carpetas por clase y cargar CSV
    csv_data = crear_directorios_mels(csv_path, mel_spec_dir)

    # 🔄 Generación de espectrogramas Mel
    print("🎛️ Generando espectrogramas Mel...")
    for _, row in csv_data.iterrows():
        nombre = row['name']
        clase = row['class']
        ruta_audio = row['audio_path']
        salida = os.path.join(mel_spec_dir, clase, nombre.split('.')[0] + '.png')
        print(f"🎵 {nombre} | Clase: {clase}")
        generar_mel_spectrogram(ruta_audio, salida)
        print("—" * 70)